In [7]:
import pandas as pd 
import seaborn as sns 
import matplotlib as plt 
%matplotlib inline
import glob
import csv
import tarfile
from tqdm import tqdm
import requests as rq
import json
from bs4 import BeautifulSoup as bs
import urllib3

In [ ]:
files = glob.glob('../BL_Work/openPrescribe/uncompressed/*.tar.gz')

In [ ]:
def readChem(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('CHEM') > 0:
                df = pd.read_csv(f,compression='infer')
                return df

In [8]:
testurl = "https://www.drugbank.ca/unearth/q?utf8=✓&searcher=drugs&query=Loperamide+Hydrochloride"
response = rq.get(testurl)
soup = bs(response.text,'html.parser')

In [26]:
A = [i for i in soup.find_all('dt') if 'Ass' in i.text]

In [77]:
dts = enumerate(soup.find('div', class_='card-content').findAll('dt'))
dds = soup.find('div', class_='card-content').findAll('dd')
dtInd = [k for (k,i) in dts if i.text == 'Associated Conditions' ][0]


In [81]:
[{i.attrs.get('href'): i.text} for i in dds[dtInd].findAll('a')]

[{u'/indications/DBCOND0089496': u'Chronic Functional Diarrhea'},
 {u'/indications/DBCOND0024414': u'Intestinal stoma leak'},
 {u'/indications/DBCOND0033890': u"Travelers' Diarrhea"},
 {u'/indications/DBCOND0102291': u'Severe or persistent diarrhea'}]

In [ ]:
drugContainers = soup.find_all('div', class_ = 'unearth-search-results unearth-indication-search-results')

In [4]:
len(drugFiles)

9318

In [ ]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=drugs&"
query = "query="
filtersFrag= "&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter"
# params={'utf8':"%E2%9C%93" , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'0','eu':'1','commit':"Apply+Filter" , 'query':''}

DiseaseDrugJson = {}

for disease in DiseaseList:
    url = urlhead + frag1 + frag2 +query+disease+ filtersFrag
#     params['query']=disease
    response = rq.get(url)
    print response.url
    soup = bs(response.text,'html.parser')
    synonym = soup.find_all('div', class_ = 'search-highlights')
    drug_matches = soup.find_all('div', class_ = 'db-matches')
    print len(drug_matches) , len(synonym)
    MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
    try:
        for i in range(len(synonym)):
            syn = synonym[i]
            drug = drug_matches[i]
            synText = syn.text.replace(u'\u2026','')
            drugsName = drug.text.split('/')
            MatchJson['Synonyms'].append(synText)
            MatchJson['MatchedDrugs'].append(drugsName)
    except:
        print "Something went horribly Wrong"
    DiseaseDrugJson[disease] = MatchJson